# <B> Setup for SageMaker Immersion Day with DL Framework </B>
* Container: conda_python3

In [1]:
import IPython
!pip install amazon-codewhisperer-jupyterlab-ext
!jupyter server extension enable amazon_codewhisperer_jupyterlab_ext

  Installing build dependencies ... |^C
anceled
ERROR: Operation cancelled by user
^C
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/bin/jupyter-server", line 6, in <module>
    from jupyter_server.serverapp import main
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/jupyter_server/serverapp.py", line 32, in <module>
    import jupyter_client
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/jupyter_client/__init__.py", line 3, in <module>
    from .asynchronous import AsyncKernelClient
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/jupyter_client/asynchronous/__init__.py", line 1, in <module>
    from .client import AsyncKernelClient  # noqa
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/jupyter_client/asynchronous/client.py", line 11, in <module>
    from ..channels import AsyncZMQSocketChannel, HBChannel
  File "/home/ec2-user/anaconda3/envs/p

In [ ]:
#create function to covert a json to

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 0. Install packages

In [4]:
install_needed = True  # should only be True once
# install_needed = False

In [5]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

Already revised


In [6]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U smdebug sagemaker-experiments
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip uninstall pycodestyle -y
    
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Found existing installation: pycodestyle 2.10.0
Uninstalling pycodestyle-2.10.0:
  Successfully uninstalled pycodestyle-2.10.0


## 1. Set roles

In [2]:
from sagemaker import get_execution_role

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[01/10/25 08:57:40] INFO     Found credentials from IAM Role:                                   ]8;id=821623;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=791262;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

                    INFO     Found credentials from IAM Role:                                   ]8;id=994678;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=691186;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20240211T155431


### 1.1 Attach IAM polich to sagemaker execution role (<b>with console</b>)
> step 1. IAM console 로 이동    
>
> step 2. 왼쪽 네비게이터에서  "Role" 선택  
> ![nn](images/Role.png)  
>
> step 3. SageMaker Execution Role 검색 후 role 선택 (상위 cell output 참조)  
> ![nn](images/search-by-rolename.png)  
>
> step 4. "attach policies" 메뉴 선택 
> ![nn](images/attach-policy-menu.png)  
>
> step 5. "IAMFullAccess" policy 검색 후 attach 
> ![nn](images/attach-policy.png) 

In [4]:
from utils.iam import iam_handler

In [5]:
iam = iam_handler()

                    INFO     Found credentials from IAM Role:                                   ]8;id=363802;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=228088;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [6]:
strID = "DJ"
strPrefix = f'{strID}-SM-PIPELINE'

In [7]:
strPrefix

'DJ-SM-PIPELINE'

### 1.2 Attach policy - sagemaker execution role
> SSM  
> S3 <BR>
> CodeCommit <BR>
> CodeBuild <BR>
> CodePipeline <BR>
> EventBridge <BR>
> Lambda <BR>
> ECR

In [8]:
listPolicyArn=[
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
    #"arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess",
    "arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess"
]
for strPolicyArn in listPolicyArn: iam.attach_policy(strSageMakerRoleName, strPolicyArn)

### 1.3 Create role for lambda

In [9]:
listService = ["lambda.amazonaws.com"]
strRoleName = "-".join([strPrefix, "LabmdaRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
    "arn:aws:iam::aws:policy/AmazonSNSFullAccess",
]
strDescription="Role for Lambda to call SageMaker functions'"

In [10]:
lambdaRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

== CREATE ROLE ==
  Role Name: [DJ-SM-PIPELINE-LabmdaRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': '5714003c-1990-4e43-9c37-98a9f98bc926', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 10 Jan 2025 08:57:43 GMT', 'x-amzn-requestid': '5714003c-1990-4e43-9c37-98a9f98bc926', 'content-type': 'text/xml', 'content-length': '212'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole
{'ResponseMetadata': {'RequestId': 'e6122175-1d1d-41e8-a872-a8826529835a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 10 Jan 2025 08:57:44 GMT', 'x-amzn-requestid': 'e6122175-1d1d-41e8-a872-a8826529835a', 'content-type': 'text/xml', 'content-length': '212'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AmazonSNSFullAccess
{'ResponseMetadata': {'RequestId': '9549226a-f744-47df-80d1-2bda35fe83c3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 10 Jan 2025 08:57:44 GMT', 'x-amzn-requestid': '9549226a-f744-4

### 1.4 role-codebuild

In [11]:
listService = ["codebuild.amazonaws.com", "sagemaker.amazonaws.com"]
strRoleName = "-".join([strPrefix, "CodeBuildRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/CloudWatchFullAccess",
    "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess",
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
]
strDescription="Role for CodeBuild"

In [12]:
buildRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)
print (f"CODE-BUILD-ROLE-ARN: {buildRoleArn}")

== CREATE ROLE ==
  Role Name: [DJ-SM-PIPELINE-CodeBuildRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': 'a87f3c7c-c2c3-4aa5-9209-53197c3c4c01', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 10 Jan 2025 08:57:46 GMT', 'x-amzn-requestid': 'a87f3c7c-c2c3-4aa5-9209-53197c3c4c01', 'content-type': 'text/xml', 'content-length': '212'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AmazonSSMFullAccess
{'ResponseMetadata': {'RequestId': '0808221b-d496-4d56-9e5b-a11d466b4aea', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 10 Jan 2025 08:57:45 GMT', 'x-amzn-requestid': '0808221b-d496-4d56-9e5b-a11d466b4aea', 'content-type': 'text/xml', 'content-length': '212'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess
{'ResponseMetadata': {'RequestId': 'f1017352-124a-4307-9fd0-75bc5752de26', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 10 Jan 2025 08:57:46 GMT', 'x-amzn-requestid': 'f1017352-124a-4307-9fd0-75bc5

### 1.5 role-codepipeline

In [13]:
listService = ["codepipeline.amazonaws.com", "codebuild.amazonaws.com"]
strRoleName = "-".join([strPrefix, "CodePipelineRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AWSCodeDeployFullAccess",
    "arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess"
]
strDescription="Role for CodePipeline"

In [14]:
pipelineRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

print (f"CODE-PIPELINE-ROLE-ARN: {pipelineRoleArn}")

== CREATE ROLE ==
  Role Name: [DJ-SM-PIPELINE-CodePipelineRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': 'be268535-f503-45f1-9da4-8c0164e2bf78', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 10 Jan 2025 08:57:49 GMT', 'x-amzn-requestid': 'be268535-f503-45f1-9da4-8c0164e2bf78', 'content-type': 'text/xml', 'content-length': '212'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess
{'ResponseMetadata': {'RequestId': '0d430a14-bcab-419f-aff5-12cb427795e1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 10 Jan 2025 08:57:48 GMT', 'x-amzn-requestid': '0d430a14-bcab-419f-aff5-12cb427795e1', 'content-type': 'text/xml', 'content-length': '212'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AWSCodeCommitFullAccess
{'ResponseMetadata': {'RequestId': '0c67cb99-f3bb-4f4c-b8b9-3b7d377e565f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 10 Jan 2025 08:57:49 GMT', 'x-amzn-requestid': '0c67cb99-f3bb-4f4c-b8b

### 1.6 role-eventbridge

In [15]:
listService = ["events.amazonaws.com"]
strRoleName = "-".join([strPrefix, "EventBridgeRole"])
listPolicyArn = [
    "arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess",
]
strDescription="Role for EventBridge"

In [16]:
eventBridgeRoleArn = iam.create_role(
    listService=listService,
    strRoleName=strRoleName,
    listPolicyArn = listPolicyArn,
    strDescription=strDescription
)

print (f"CODE-PIPELINE-ROLE-ARN: {eventBridgeRoleArn}")

== CREATE ROLE ==
  Role Name: [DJ-SM-PIPELINE-EventBridgeRole] is already exist!!, so, this will be deleted and re-created.
{'ResponseMetadata': {'RequestId': '324dfae3-f640-41d7-a3c6-a40f8bd90b3f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 10 Jan 2025 08:57:51 GMT', 'x-amzn-requestid': '324dfae3-f640-41d7-a3c6-a40f8bd90b3f', 'content-type': 'text/xml', 'content-length': '212'}, 'RetryAttempts': 0}}
arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess
  Service name: ['events.amazonaws.com'], 
  Role name: DJ-SM-PIPELINE-EventBridgeRole, 
  Policys: ['arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess'], 
  Role ARN: arn:aws:iam::419974056037:role/DJ-SM-PIPELINE-EventBridgeRole
== COMPLETED ==
CODE-PIPELINE-ROLE-ARN: arn:aws:iam::419974056037:role/DJ-SM-PIPELINE-EventBridgeRole


## 2. Set datasets and utils

In [17]:
import os
import boto3
import sagemaker
import pandas as pd
from utils.s3 import s3_handler
from utils.ssm import parameter_store

### 2.1 download and manipulate data

In [18]:
s3 = boto3.client("s3")

In [19]:
strDataPath = os.path.join("./dataset", "abalone.csv")
s3.download_file("sagemaker-sample-files", f"datasets/tabular/uci_abalone/abalone.csv", strDataPath)

listFeatureName = [
    "Sex",
    "Length",
    "Diameter",
    "Height",
    "Whole weight",
    "Shucked weight",
    "Viscera weight",
    "Shell weight",
    "Rings",
]
pdData = pd.read_csv(strDataPath, header=None, names=listFeatureName)

# SageMaker XGBoost has the convention of label in the first column
pdData = pdData[listFeatureName[-1:] + listFeatureName[:-1]]
pdData["Sex"] = pdData["Sex"].astype("category").cat.codes
pdData.to_csv(strDataPath, header=True, index=False)

### 2.2 upload to S3 and regiter path to parameter store

In [20]:
strRegionName=boto3.Session().region_name
s3 = s3_handler()
pm = parameter_store(strRegionName)

This is a S3 handler with [None] region.


In [21]:
strBucketName = sagemaker.Session().default_bucket()

In [22]:
source_dir, target_bucket, target_dir = os.path.dirname(strDataPath), strBucketName, "-".join([strPrefix, "DATA"])
s3.upload_dir(source_dir, target_bucket, target_dir)
pm.put_params(key="-".join([strPrefix, "DATA-PATH"]), value=f"s3://{target_bucket}/{target_dir}", overwrite=True)

Upload:[./dataset] was uploaded to [s3://sagemaker-us-west-2-419974056037/DJ-SM-PIPELINE-DATA]successfully


'Store suceess'

### 2.3 Copy utils to pipeline/utils

In [23]:
from distutils.dir_util import copy_tree

In [24]:
copy_tree("./utils", "./mlops/pipeline/utils")

[01/10/25 08:57:54] INFO     copying ./utils/ecr.py -> ./mlops/pipeline/utils                      ]8;id=111516;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/file_util.py\file_util.py]8;;\:]8;id=149680;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/file_util.py#137\137]8;;\

                    INFO     copying ./utils/s3.py -> ./mlops/pipeline/utils                       ]8;id=660341;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/file_util.py\file_util.py]8;;\:]8;id=225870;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/file_util.py#137\137]8;;\

                    INFO     creating mlops/pipeline/utils/cicd_templates                            ]8;id=963143;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/dir_util.py\dir_util.py]8;;\:]8;id=251437;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/dir_util.py#71\71]8;;\

                    INFO     copying ./utils/cicd_templates/code_build_template.json ->            ]8;id=539833;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/file_util.py\file_util.py]8;;\:]8;id=252490;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/file_util.py#137\137]8;;\
                             ./mlops/pipeline/utils/cicd_templates                                                 

                    INFO     copying ./utils/cicd_templates/code_pipeline_template.json ->         ]8;id=715613;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/file_util.py\file_util.py]8;;\:]8;id=895588;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/file_util.py#137\137]8;;\
                             ./mlops/pipeline/utils/cicd_templates                                                 

...

                    INFO     creating mlops/pipeline/utils/__pycache__                               ]8;id=235968;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/dir_util.py\dir_util.py]8;;\:]8;id=123032;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/dir_util.py#71\71]8;;\

                    INFO     copying ./utils/__pycache__/s3.cpython-310.pyc ->                     ]8;id=549299;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/file_util.py\file_util.py]8;;\:]8;id=518354;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/file_util.py#137\137]8;;\
                             ./mlops/pipeline/utils/__pycache__                                                    

                    INFO     copying ./utils/__pycache__/iam.cpython-310.pyc ->                    ]8;id=348585;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/file_util.py\file_util.py]8;;\:]8;id=356370;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/file_util.py#137\137]8;;\
                             ./mlops/pipeline/utils/__pycache__                                                    

                    INFO     copying ./utils/__pycache__/ssm.cpython-310.pyc ->                    ]8;id=290905;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/file_util.py\file_util.py]8;;\:]8;id=884735;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/file_util.py#137\137]8;;\
                             ./mlops/pipeline/utils/__pycache__                                                    

['./mlops/pipeline/utils/ecr.py',
 './mlops/pipeline/utils/s3.py',
 './mlops/pipeline/utils/cicd_templates/code_build_template.json',
 './mlops/pipeline/utils/cicd_templates/code_pipeline_template.json',
 './mlops/pipeline/utils/iam.py',
 './mlops/pipeline/utils/code_pipeline.json',
 './mlops/pipeline/utils/code_pipeline.py',
 './mlops/pipeline/utils/code_build.json',
 './mlops/pipeline/utils/monitoringjob_utils.py',
 './mlops/pipeline/utils/lambda_func.py',
 './mlops/pipeline/utils/ssm.py',
 './mlops/pipeline/utils/__pycache__/s3.cpython-310.pyc',
 './mlops/pipeline/utils/__pycache__/iam.cpython-310.pyc',
 './mlops/pipeline/utils/__pycache__/ssm.cpython-310.pyc']

## 3. Create custom docker image

In [25]:
from utils.ecr import ecr_handler
ecr = ecr_handler()
strAccountID = boto3.client("sts").get_caller_identity().get("Account")

### 3.1 For preprocessing

In [26]:
%%writefile custom-docker/Dockerfile-prep

FROM 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:1.0-1-cpu-py3
RUN pip install -U pip
RUN pip install -U sagemaker
RUN pip3 install pyathena awswrangler 
ENV PYTHONUNBUFFERED=TRUE

Overwriting custom-docker/Dockerfile-prep


In [27]:
strRepositoryName="mlops-image-prep"  ## <-- 원하는 docker repostory 이름을 추가
strRepositoryName = strRepositoryName.lower()
strDockerFile = "Dockerfile-prep"
strDockerDir = "./custom-docker/"
strTag = "latest"

In [28]:
ecr.build_docker(strDockerDir, strDockerFile, strRepositoryName, strRegionName="ap-northeast-2", strAccountId="366743142698")

/home/ec2-user/SageMaker/mlops-step-alert/2.building-pipelines
/home/ec2-user/SageMaker/mlops-step-alert/2.building-pipelines/custom-docker
strDockerFile Dockerfile-prep
aws ecr get-login --region 'ap-northeast-2' --registry-ids '366743142698' --no-include-email


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded




KeyboardInterrupt



In [ ]:
strEcrRepositoryUri = ecr.register_image_to_ecr(strRegionName, strAccountID, strRepositoryName, strTag)

### 3.2 For training

In [55]:
%%writefile custom-docker/Dockerfile-tr

FROM 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-xgboost:1.5-1
RUN pip install -U pip
RUN pip install -U sagemaker
RUN pip install shap
RUN pip install sagemaker-inference
RUN pip install seaborn==0.11.2
ENV PATH="/opt/ml/code:${PATH}"
ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code

# /opt/ml and all subdirectories are utilized by SageMaker, 
# use the /code subdirectory to store your user code.
#COPY train.py /opt/ml/code/train.py

# Defines train.py as script entrypoint
#ENV SAGEMAKER_PROGRAM train.py
ENV PYTHONUNBUFFERED=TRUE

Overwriting custom-docker/Dockerfile-tr


In [56]:
strTrRepositoryName="mlops-image-tr"  ## <-- 원하는 docker repostory 이름을 추가
strTrRepositoryName = strTrRepositoryName.lower()
strDockerFile = "Dockerfile-tr"
strDockerDir = "./custom-docker/"
strTag = "latest"

In [57]:
ecr.build_docker(strDockerDir, strDockerFile, strTrRepositoryName, strRegionName="ap-northeast-2", strAccountId="366743142698")

/home/ec2-user/SageMaker/mlops-step-alert/2.building-pipelines
/home/ec2-user/SageMaker/mlops-step-alert/2.building-pipelines/custom-docker
strDockerFile Dockerfile-tr
aws ecr get-login --region 'ap-northeast-2' --registry-ids '366743142698' --no-include-email


WARNING! Using --password via the CLI is insecure. Use --password-stdin.


Login Succeeded

Sending build context to Docker daemon  7.168kB

Step 1/9 : FROM 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-xgboost:1.5-1
 ---> 2dee2c955721
Step 2/9 : RUN pip install -U pip
 ---> Using cache
 ---> 657d6c353448
Step 3/9 : RUN pip install -U sagemaker
 ---> Using cache
 ---> 6018b8f1f76f
Step 4/9 : RUN pip install shap
 ---> Using cache
 ---> dd58906f9e8c
Step 5/9 : RUN pip install sagemaker-inference
 ---> Using cache
 ---> dd416b9ce3a3
Step 6/9 : RUN pip install seaborn==0.11.2
 ---> Using cache
 ---> 250208a89bcb
Step 7/9 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 6d4f3d1f4ad9
Step 8/9 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
 ---> Using cache
 ---> e74595f60710
Step 9/9 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 4686be875f2e
Successfully built 4686be875f2e
Successfully tagged mlops-image-tr:latest

/home/ec2-user/SageMaker/mlops-step-alert/2.building-pipelines


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [58]:
strTrEcrRepositoryUri = ecr.register_image_to_ecr(strRegionName, strAccountID, strTrRepositoryName, strTag)

== REGISTER AN IMAGE TO ECR ==
  processing_repository_uri: 419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-tr:latest
aws ecr get-login --region 'us-east-1' --registry-ids '419974056037' --no-include-email


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded

aws ecr create-repository --repository-name 'mlops-image-tr'
docker tag 'mlops-image-tr:latest' '419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-tr:latest'
docker push '419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-tr:latest'
== REGISTER AN IMAGE TO ECR ==


## 4. Set default parameters

In [59]:
pm.put_params(key="PREFIX", value=strPrefix, overwrite=True)
pm.put_params(key="-".join([strPrefix, "REGION"]), value=strRegionName, overwrite=True)
pm.put_params(key="-".join([strPrefix, "BUCKET"]), value=strBucketName, overwrite=True)
pm.put_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]), value=get_execution_role(), overwrite=True)
pm.put_params(key="-".join([strPrefix, "ACCOUNT-ID"]), value=strAccountID, overwrite=True)
pm.put_params(key="-".join([strPrefix, "IMAGE-URI-PREP"]), value=strEcrRepositoryUri, overwrite=True)
pm.put_params(key="-".join([strPrefix, "IMAGE-URI-TR"]), value=strTrEcrRepositoryUri, overwrite=True)
pm.put_params(key="-".join([strPrefix, "LAMBDA-ROLE-ARN"]), value=lambdaRoleArn, overwrite=True)
pm.put_params(key="-".join([strPrefix, "CODE-BUILD-ROLE-ARN"]), value=buildRoleArn, overwrite=True)
pm.put_params(key="-".join([strPrefix, "CODE-PIPELINE-ROLE-ARN"]), value=pipelineRoleArn, overwrite=True)
pm.put_params(key="-".join([strPrefix, "CODE-EVENTBRIDGE-ROLE-ARN"]), value=eventBridgeRoleArn, overwrite=True)

'Store suceess'

In [60]:
print ("PREFIX: ", pm.get_params(key="PREFIX"))
print ("REGION: ", pm.get_params(key="-".join([strPrefix, "REGION"])))
print ("BUCKET: ", pm.get_params(key="-".join([strPrefix, "BUCKET"])))
print ("SAGEMAKER-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"])))
print ("ACCOUNT-ID: ", pm.get_params(key="-".join([strPrefix, "ACCOUNT-ID"])))
print ("IMAGE-URI-PREP: ", pm.get_params(key="-".join([strPrefix, "IMAGE-URI-PREP"])))
print ("IMAGE-URI-TR: ", pm.get_params(key="-".join([strPrefix, "IMAGE-URI-TR"])))
print ("LAMBDA-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "LAMBDA-ROLE-ARN"])))
print ("CODE-BUILD-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-BUILD-ROLE-ARN"])))
print ("CODE-PIPELINE-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-PIPELINE-ROLE-ARN"])))
print ("CODE-EVENTBRIDGE-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-EVENTBRIDGE-ROLE-ARN"])))
print ("DATA-PATH: ", pm.get_params(key="-".join([strPrefix, "DATA-PATH"])))

PREFIX:  DJ-SM-PIPELINE
REGION:  us-east-1
BUCKET:  sagemaker-us-east-1-419974056037
SAGEMAKER-ROLE-ARN:  arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436
ACCOUNT-ID:  419974056037
IMAGE-URI-PREP:  419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-prep:latest
IMAGE-URI-TR:  419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-tr:latest
LAMBDA-ROLE-ARN:  arn:aws:iam::419974056037:role/DJ-SM-PIPELINE-LabmdaRole
CODE-BUILD-ROLE-ARN:  arn:aws:iam::419974056037:role/DJ-SM-PIPELINE-CodeBuildRole
CODE-PIPELINE-ROLE-ARN:  arn:aws:iam::419974056037:role/DJ-SM-PIPELINE-CodePipelineRole
CODE-EVENTBRIDGE-ROLE-ARN:  arn:aws:iam::419974056037:role/DJ-SM-PIPELINE-EventBridgeRole
DATA-PATH:  s3://sagemaker-us-east-1-419974056037/DJ-SM-PIPELINE-DATA


In [47]:
container = sagemaker.image_uris.retrieve("xgboost", strRegionName, "1.7-1")

In [48]:
container

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1'